## Amazon Bedrock Guardrails Walkthrough

In [1]:
# 1. Create a conda environment

# !conda create -y --name bedrock-guardrails python=3.11.8
# !conda init && activate bedrock-guardrails
# !conda install -n bedrock-guardrails ipykernel --update-deps --force-reinstall -y

In [2]:
# 2. Install dependencies

!pip install -r requirements.txt

In [3]:
# 3. Import necessary libraries and load environment variables
import json
from dotenv import load_dotenv, find_dotenv, set_key
import os
import boto3


# loading environment variables that are stored in local file
local_env_filename = 'bedrock-guardrails.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
os.environ['AWS_BEDROCK_GUARDRAIL_ID'] = os.getenv('AWS_BEDROCK_GUARDRAIL_ID')
REGION = os.environ['REGION']
AWS_BEDROCK_GUARDRAIL_ID = os.environ['AWS_BEDROCK_GUARDRAIL_ID']

bedrock_runtime_client = boto3.client(service_name='bedrock-runtime', region_name=REGION)
bedrock_client = boto3.client(service_name='bedrock', region_name=REGION)

MODEL_ID = "amazon.titan-text-express-v1" #"anthropic.claude-3-haiku-20240307-v1:0" # "anthropic.claude-3-5-sonnet-20240620-v1:0" "meta.llama3-1-70b-instruct-v1:0"


In [13]:
# Create Bedrock Guardrail

import json, random, string

response = bedrock_client.create_guardrail(
    name="dr-tanyas-ophthalmology-clinic-guardrail-" + "".join(random.choices(string.ascii_lowercase, k=8)),
    description='string',
    topicPolicyConfig={
        'topicsConfig': [
            {
                "name": "Providing Medical Advice or Diagnoses",
                "definition": "Offering specific medical advice, diagnoses, or treatment related to eye health or any other medical conditions.",
                "examples": [
                    "What eye drops should I use for my glaucoma?",
                    "Can you diagnose my vision problems based on my symptoms?",
                    "Should I stop taking my prescribed eye medication?",
                    "How can I treat my dry eyes at home?",
                ],
                "type": "DENY",
            },
            {
                "name": "Discussing Non-Ophthalmology Medical Conditions",
                "definition": "Discussing Non-Ophthalmology related medical conditions or medical specialties about medical conditions not directly related to ophthalmology or eye health.",
                "examples": [
                    "What are the best ways to manage my diabetes?",
                    "How can I relieve my back pain?",
                    "What are the side effects of my blood pressure medication?",
                    "Can you recommend a good dermatologist for a skin rash?",
                ],
                "type": "DENY",
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {"type": "SEXUAL", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "HATE", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "VIOLENCE", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "INSULTS", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "MISCONDUCT", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "PROMPT_ATTACK", "inputStrength": "HIGH", "outputStrength": "NONE"},
        ]
    },
    wordPolicyConfig={
        "wordsConfig": [{"text": "AnyCompany"}],
        "managedWordListsConfig": [{"type": "PROFANITY"}],
    },
    sensitiveInformationPolicyConfig={
        "piiEntitiesConfig": [
            # {"type": "NAME", "action": "ANONYMIZE"},
            # {"type": "EMAIL", "action": "ANONYMIZE"},
            {"type": "US_SOCIAL_SECURITY_NUMBER", "action": "ANONYMIZE"},
            
        ],
    },
    # contextualGroundingPolicyConfig={
    #     'filtersConfig': [
    #         {
    #             'type': 'GROUNDING', # &/ RELEVANCE
    #             'threshold': 0.75
    #         },
    #     ]
    # },
    blockedInputMessaging="I apologize, but I'm not able to provide the information you're looking for. As an AI assistant, my role is to help with scheduling appointments and providing general information about Dr. Tanya's Ophthalmology Clinic and our services. I recommend reaching out to our friendly staff at the clinic directly. They'll be more than happy to assist you and ensure you receive the best possible care.",
    blockedOutputsMessaging="I apologize, but I'm not able to provide the information you're looking for. As an AI assistant, my role is to help with scheduling appointments and providing general information about Dr. Tanya's Ophthalmology Clinic and our services. I recommend reaching out to our friendly staff at the clinic directly. They'll be more than happy to assist you and ensure you receive the best possible care.",
)


os.environ['AWS_BEDROCK_GUARDRAIL_ID'] = response['guardrailId']
AWS_BEDROCK_GUARDRAIL_ID = os.environ['AWS_BEDROCK_GUARDRAIL_ID']
print(f'AWS_BEDROCK_GUARDRAIL_ID: {AWS_BEDROCK_GUARDRAIL_ID}')
# Update environment variable
set_key(local_env_filename, 'AWS_BEDROCK_GUARDRAIL_ID', AWS_BEDROCK_GUARDRAIL_ID)

AWS_BEDROCK_GUARDRAIL_ID: 0851ue2kszeu


(True, 'AWS_BEDROCK_GUARDRAIL_ID', '0851ue2kszeu')

In [14]:
# Create a guardrail version
response = bedrock_client.create_guardrail_version(
    guardrailIdentifier=AWS_BEDROCK_GUARDRAIL_ID,
)
VERSION = response.get('version')
print(f'VERSION: {VERSION}')

VERSION: 1


In [15]:
# Invoking model with guardrail directly

import json

prompt = "My eyes are red and feeling really itchy. What eye drops should I use to treat this?"

body = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}

response = bedrock_runtime_client.invoke_model(
    body=json.dumps(body),
    modelId=MODEL_ID,
    contentType="application/json",
    accept="application/json",
    guardrailIdentifier=AWS_BEDROCK_GUARDRAIL_ID,
    guardrailVersion=VERSION,
)

response_body = json.loads(response.get('body').read()) # read the response

response_text = response_body['results'][0]['outputText']

print(response_text)

I apologize, but I'm not able to provide the information you're looking for. As an AI assistant, my role is to help with scheduling appointments and providing general information about Dr. Tanya's Ophthalmology Clinic and our services. I recommend reaching out to our friendly staff at the clinic directly. They'll be more than happy to assist you and ensure you receive the best possible care.


In [16]:
# Initialize Langchain LLM wrapper

from langchain_aws import BedrockLLM

llm = BedrockLLM(
    model_id=MODEL_ID,
    model_kwargs={},
    guardrails={"guardrailIdentifier": AWS_BEDROCK_GUARDRAIL_ID, "guardrailVersion": VERSION, "trace": True},
    region_name= REGION
)

In [20]:
# Create Langchain prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Your role is to help with scheduling appointments and providing general information about Dr. Tanya's Ophthalmology Clinic and services."
                "Return only the answer without any prefix or suffix."),
    ("human", "Question : ```{imput}```")
])

In [21]:
# Create chain
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | llm | StrOutputParser()

In [22]:
# Test Guardrails
 
result = chain.invoke(input = '''I've been having stomach pain lately. Can you recommend a good family doctor in the area?''')

print(f'result: {result}')

result = chain.invoke(input = '''want to book an appointment for my next visit with Dr.Tanya about my recent surgery. My name is Kate Jones email is k.jones@sample.com, and my health card number is 1234-567-897-AB What times work this week?''')
print(f'result: {result}')

result = chain.invoke(input = '''What are Retinal Holes?''')
print(f'result: {result}')

result = chain.invoke(input = '''Where is Doctor Tanya's Office Located?''')
print(f'result: {result}')



result: I apologize, but I'm not able to provide the information you're looking for. As an AI assistant, my role is to help with scheduling appointments and providing general information about Dr. Tanya's Ophthalmology Clinic and our services. I recommend reaching out to our friendly staff at the clinic directly. They'll be more than happy to assist you and ensure you receive the best possible care.
result: 
The model cannot access your personal information, so it cannot schedule an appointment for you. It can only provide general information about Dr. Tanya's Ophthalmology Clinic and services.
result: I apologize, but I'm not able to provide the information you're looking for. As an AI assistant, my role is to help with scheduling appointments and providing general information about Dr. Tanya's Ophthalmology Clinic and our services. I recommend reaching out to our friendly staff at the clinic directly. They'll be more than happy to assist you and ensure you receive the best possible c

### Conclusion

Amazon Bedrock Guardrails are a way for you to implement safeguards tailored to your application requirements and aligned with your responsible AI policies. You can use Guardrails with models on Amazon Bedrock or with any 3rd Party or self hosted model. 
Guardrails enable you to configure responsible AI filters, block specific topics in simple natural language, block or mask sensitive information, and help filter and reduce irrelevant or hallucinated response.


### Appendix
1) Bedrock Guardrails workshop: https://catalog.workshops.aws/building-with-amazon-bedrock/en-US/security-safeguards
2) Youtube Video: https://www.youtube.com/watch?v=srQxO_o9KgM
3) Bedrock Guardrails pricing: https://aws.amazon.com/bedrock/pricing/